# Monadster

Python implementation of Scott Wlaschin's F# script demonstrating a custom state monad 

**References**

GitHub Gist: https://gist.github.com/swlaschin/54489d9586402e5b1e8a

Blog Post: http://fsharpforfunandprofit.com/posts/monadster/

## Types helpers

Import some Python modules to help us define type classes.

For Python 2.7 we use the `attrs` library automate generation of boiler-plate code for classes that store plain data.


In [1]:
#
# PYTHON 2.7: attrs
# 

import attr

# product_type
data_type = attr.s

# field
data_field = attr.ib
# field_type
data_field_type = attr.validators.instance_of

Python does not have builtin Algebraic Data Types (`product types` and `sum types`). 

Fortunately the `sumtypes` module can help us with this.

First we need to `pip install sumtypes` 

In [2]:
!pip install sumtypes

In [3]:
#
# PYTHON 2.7: sumtypes
# 

import sumtypes

# sum_type
abstract_type = sumtypes.sumtype

# match matching
data_match = sumtypes.match

For Python 3.6 there are some other Python modules that can also help.

**dataclasses** module

> https://hackernoon.com/a-brief-tour-of-python-3-7-data-classes-22ee5e046517

**typing** module

> https://medium.com/@ageitgey/learn-how-to-use-static-type-checking-in-python-3-6-in-10-minutes-12c86d72677b



Python does not have currying of functions by default.

Nor does Python have built-in pipable functions.

Fortunately the `toolz` library can provide both.  

In [4]:
#
# PYTHON 2.7: toolz
# 

from toolz import curry
from toolz import pipe

In [5]:
# =================================================================================
# The Common Context (Label,VitalForce)
# =================================================================================

# All body parts have a label
@data_type
class Label(object):
    label = data_field(validator=data_field_type(str))

# The Animal Electricity needed to create a live part
@data_type
class VitalForce(object):
    units = data_field(validator=data_field_type(int))

# get one unit of vital force and return the unit and the remaining
def getVitalForce(vitalForce): 
    oneUnit = VitalForce(1)
    remaining = VitalForce(vitalForce.units - 1)  # decrement
    return oneUnit, remaining  # return both


In [6]:
# =================================================================================
# The Left Leg 
# =================================================================================

# Dr Frankenfunctor has a dead left leg lying around in the lab
@data_type
class DeadLeftLeg(object):
    label = data_field(validator=data_field_type(Label))

# and can make a live left leg from it
@data_type
class LiveLeftLeg(object):
    label = data_field(validator=data_field_type(Label))
    vitalForce = data_field(validator=data_field_type(VitalForce))

In [7]:
# how to make a live thing?  -- First approach
# module Approach1 = ... 

In [8]:
# version 2 -- the input is a curried version. 
# module Approach2 = ...

In [9]:
# version 3 -- the input is a DeadLeftLeg, returning a generator function
# module Approach3 = ...

In [10]:
# Demonstrates how currying works
# module CurryingExample = ...

In [11]:
# =================================================================================
# Creating the Monadster type
# =================================================================================

# version 4 -- make the function generic
# module Approach4 = ...

In [12]:
# final version -- wrap Monadster body part recipe with "M"
#type M<'LiveBodyPart> = 
#    M of (VitalForce -> 'LiveBodyPart * VitalForce)
@data_type
class M(object):
    m = data_field()  # can contain any type
          
# the creation function looks like this now
def makeLiveLeftLegM(deadLeftLeg):
    def becomeAlive(vitalForce):
        label = deadLeftLeg.label
        oneUnit, remainingVitalForce = getVitalForce(vitalForce)
        liveLeftLeg = LiveLeftLeg(label, oneUnit)
        return liveLeftLeg, remainingVitalForce    
    return M(becomeAlive)  # wrap the function in a single case union

# and the function signature is:
# val makeLiveLeftLegM : DeadLeftLeg -> M<LiveLeftLeg>

In [13]:
# ---------------------------------------------------------------------------------
# Testing the left leg
# ---------------------------------------------------------------------------------

# create Left Leg
deadLeftLeg = DeadLeftLeg(Label("Boris"))
leftLegM = makeLiveLeftLegM(deadLeftLeg)

In [14]:
# pretend that vital force is available
vf = VitalForce(10)

innerFn = leftLegM.m
liveLeftLeg, remainingAfterLeftLeg = innerFn(vf)
# val liveLeftLeg : LiveLeftLeg = 
#    LiveLeftLeg ("Boris",{units = 1;})
# val remainingAfterLeftLeg : VitalForce = 
#    {units = 9;}

In [15]:
# encapsulate the function call that "runs" the recipe
def runM(f, vitalForce):
    return f.m(vitalForce)

liveLeftLeg2, remainingAfterLeftLeg2 = runM(leftLegM, vf)

In [16]:
liveLeftLeg2, remainingAfterLeftLeg2

(LiveLeftLeg(label=Label(label='Boris'), vitalForce=VitalForce(units=1)),
 VitalForce(units=9))

In [17]:
# =================================================================================
# The Right Leg
# =================================================================================

# no right legs were available -- see the definition of LiveBody later for the workaround

In [18]:
# =================================================================================
# The Left Arm
# =================================================================================

# Dr Frankenfunctor has a dead but broken left arm lying around in the lab
# type DeadLeftBrokenArm = DeadLeftBrokenArm of Label 
@data_type
class DeadLeftBrokenArm(object):
    label = data_field(validator=data_field_type(Label))

# You can have a live version of the broken arm too.
@data_type
class LiveLeftBrokenArm(object):
    label = data_field(validator=data_field_type(Label))
    vitalForce = data_field(validator=data_field_type(VitalForce))
    
# There is a live version of a heathly arm, but no dead version
@data_type
class LiveLeftArm(object):
    label = data_field(validator=data_field_type(Label))
    vitalForce = data_field(validator=data_field_type(VitalForce))

# However, Dr Frankenfunctor CAN turn a broken left arm into a heathly left arm
# type HealBrokenArm = LiveLeftBrokenArm -> LiveLeftArm 

# implementation of HealBrokenArm 
def healBrokenArm(liveLeftBrokenArm):
    return LiveLeftArm(liveLeftBrokenArm.label,
                       liveLeftBrokenArm.vitalForce)

In [19]:
# version 1 - explicit, hard-coded arm type
# module HealArm_v1 = ...

In [20]:
# ---------------------------------------------------------------------------------
# Introducing mapM
# ---------------------------------------------------------------------------------

# A generic map that works with ANY body part
@curry
def mapM(f, bodyPartM): 
    def transformWhileAlive(vitalForce): 
        bodyPart, remainingVitalForce = runM(bodyPartM, vitalForce) 
        updatedBodyPart = f(bodyPart)
        return updatedBodyPart, remainingVitalForce
    return M(transformWhileAlive) 

# signature
# mapM : f:('a -> 'b) -> M<'a> -> M<'b>

# so final version is simple
healBrokenArmM = mapM(healBrokenArm)

In [21]:
# ---------------------------------------------------------------------------------
# The importance of mapM
# ---------------------------------------------------------------------------------

# some examples of map
# module TheImportanceOfMap = ...

# conversely, mapM will work with ANY normal type
# module MapMWorksWithAllTypes = ...

In [22]:
# ---------------------------------------------------------------------------------
# Testing the left arm
# ---------------------------------------------------------------------------------

def makeLiveLeftBrokenArm(deadLeftBrokenArm): 
    label = deadLeftBrokenArm.label
    def becomeAlive(vitalForce): 
        oneUnit, remainingVitalForce = getVitalForce(vitalForce) 
        liveLeftBrokenArm = LiveLeftBrokenArm(label, oneUnit)
        return liveLeftBrokenArm, remainingVitalForce    
    return M(becomeAlive)

# create a dead Left Broken Arm
deadLeftBrokenArm = DeadLeftBrokenArm(Label("Victor"))

# create a M<BrokenLeftArm> from the dead one
leftBrokenArmM = makeLiveLeftBrokenArm(deadLeftBrokenArm) 

# create a M<LeftArm> using mapM and healBrokenArm 
# let leftArmM = leftBrokenArmM  |> mapM healBrokenArm 
leftArmM = pipe(leftBrokenArmM, mapM(healBrokenArm))


# now we can run it with the vital force
# let vf = {units = 10}
liveLeftArm, remainingAfterLeftArm = runM(leftArmM, vf)
# val liveLeftArm : LiveLeftArm = LiveLeftArm ("Victor",{units = 1;})
# val remainingAfterLeftArm : VitalForce = {units = 9;}

In [23]:
liveLeftArm, remainingAfterLeftArm

(LiveLeftArm(label=Label(label='Victor'), vitalForce=VitalForce(units=1)),
 VitalForce(units=9))